# Imports

In [2]:
import cv2
import numpy as np

from os import listdir
from os.path import isfile, join

# Constants

In [2]:
# Loading the pre-trained classifier
face_classifier = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')

# This is where the trainig data will be stored
data_path = '/faces/user/'

SAMPLES = 100

# Step 1:- Create Training Data

**We will initialize the web-cam to get 100 trainig samples**

#### i) Funtion to Detect and Return Cropped Faces

In [3]:
def face_extractor(img):
    # If no face detected, it returns the input image
    
    # Convert the image into grey scale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    # Pass the grey scale image into the classifier
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

<>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-3-5b2160b436c1>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


#### ii) Initialize Webcam


In [4]:
cap = cv2.VideoCapture(0)
count = 0

#### iii) Collect 100 Trainig Samples from the webcam

In [5]:
# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        
        # resize the training image
        face = cv2.resize(face_extractor(frame), (200, 200))
        
        # Convert the training image into grey scale
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save the captured file in specified directory with unique name
        file_name_path = './faces/user/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    # Press "Enter" to exit(13 is the Enter Key)
    if cv2.waitKey(1) == 13 or count == SAMPLES:
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

error: OpenCV(4.0.1) C:\ci\opencv-suite_1573470242804\work\modules\objdetect\src\cascadedetect.cpp:1658: error: (-215:Assertion failed) !empty() in function 'cv::CascadeClassifier::detectMultiScale'


# Step 2:- Train the Model

**We will use the Model that comes pre-installed in OpenCV**

#### i) Load the Trainig data and convert it into an array

In [ ]:
# Load the training data and store the file names in a variable
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create empty arrays for training data and labels
Training_Data, Labels = [], []

# Loop over the file names of the training data
for i, files in enumerate(onlyfiles):
    
    # Join file path to file name
    image_path = data_path + onlyfiles[i]
    
    # Read every image and convert it to grey scale
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Now append these images and labels to the empty arrays
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

#### ii) Load the Pre-Installed Model

In [4]:
model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()

AttributeError: module 'cv2.face' has no attribute 'createLBPHFaceRecognizer'

#### iii) Train the Model

In [ ]:
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")

# Step 3 :- Run Facial Recognition

#### i) Initialize Web-Cam

In [ ]:
cap = cv2.VideoCapture(0)

#### ii) Function to get Face from the web-cam 

In [ ]:
def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi

#### iii) Loop over the input from the webcam and feed it into the model

In [ ]:
while True:

    # Reading frames from the webcam
    ret, frame = cap.read()
    
    # Feeding the frames into the above funtion to get face from it
    image, face = face_detector(frame)
    
    try:
        # Convert to grey scale
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass the grey scale image to the model. "results" is a tuple containing the label and confidence value
        results = model.predict(face)
        
        if results[1] < 500:
            
            # Calculate the probability of face match and display it on the screen
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 75:
            cv2.putText(image, "Face Matched", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
        else:
            cv2.putText(image, "Face not Recognised", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "Face not Recognised", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()